# LLM humor detection with Subspace based metric

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
    output_hidden_states=True  # Enable hidden states output
)

/home/pedro/Documents/humor/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:777: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [14]:
INSTRUCTIONS = [
    "This is a standup comedy transcript. Enumerate the humuorous parts of the text.",
    "You are an expert humor detector. List the funny parts in the following transcript:",
    "Detect the funny parts in the following standup comedy transcript.",
    "Find the humorous parts in the following text."
]

TRANSCRIPTS = [
    "When I was a kid, I used to fantasize about getting older, growing up and having money, and buying my mom nice things for her birthday. When I was a kid, we were poor. So poor I remember, just so I could go to my senior prom, just so I could go to my senior prom, I had to sell my U.S. passport on the street. Sold my passport on the street for 300 bucks to get to go to my prom. Of course this was before 9/11 so… my bad, everybody. Weird joke to clap for, but sure. My mom actually should’ve been on one of the planes that crashed on 9/11. I think.I mean, don’t get me wrong, I loved my mother. She was my mom, of course I loved her. We fought a lot. My mom could be very racist. Very racist. And I do not tolerate racism. That’s ignorance and I hate that. When I was a kid, like nine years old, I’d come home after school. Once in a while, I’d bring a friend over to play with me at my house. Once in a while, I’d bring a black friend over. And when I did that, my mom would act weird. She’d pull me aside and say, “Anthony, who’s your new friend? Is he a drug dealer?” And I would say, “Shut up, Mom, that’s racist. Put your money away.”I never get to see my family anymore, really. Most of them are in jail, to be honest. And we never talk, write letters or any of that shit. ‘Cause they’re all in jail for the exact same thing. My testimony."
]

GROUND_TRUTHS = [
    "So poor I remember, just so I could go to my senior prom, just so I could go to my senior prom, I had to sell my U.S. passport on the street.",
    "Of course this was before 9/11 so... my bad, everybody.",
    "Weird joke to clap for, but sure.",
    "My mom actually should've been on one of the planes that crashed on 9/11. I think.",
    "She'd pull me aside and say, ""Anthony, who's your new friend? Is he a drug dealer?""",
    "And I would say, ""Shut up, Mom, that's racist. Put your money away.",
    "Cause they're all in jail for the exact same thing. My testimony."
]

In [15]:
from itertools import product

all_representations = []
for inst, text in product(INSTRUCTIONS, TRANSCRIPTS):
    sentence = f"{inst}\n{text}\n" + "\n -".join(GROUND_TRUTHS)
    inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model(**inputs)

    representation = torch.cat(outputs.hidden_states)[-1, -16:].flatten()
    all_representations.append(representation)

In [23]:
gt_references = torch.stack(all_representations)

*_, gt_reference_subspace = torch.pca_lowrank(gt_references.float(), q=3)

gt_reference_subspace.shape

torch.Size([57344, 3])

In [38]:
sentences = [f"{inst}\n{text}\n" for inst, text in product(INSTRUCTIONS, TRANSCRIPTS)]

inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True).to(model.device)
with torch.inference_mode():
    outputs = model(input_ids=model.generate(**inputs, max_new_tokens=128))


# representations = torch.cat(outputs.hidden_states)[-1, -16:].flatten()
# all_representations.append(representation)

OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 23.55 GiB of which 18.12 MiB is free. Including non-PyTorch memory, this process has 23.51 GiB memory in use. Of the allocated memory 22.92 GiB is allocated by PyTorch, and 104.35 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [35]:
outputs.hidden_states[-1][-1].shape

torch.Size([4, 1, 3584])

In [ ]:


# Run all questions with the model
for instruction in INSTRUCTIONS:
    result = llama_pipeline(instruction + "\n" + TRANSCRIPTS)
    print(f"Instruction: {instruction}")
    print(f"Result: {result[0]['generated_text']}\n")